### MA6202: Laboratorio de Ciencia de Datos

#### Profesor: Nicolás Caro

#### Fecha de entrega: 17/05/2020

#### Integrantes: Matías Romero, Danner Schlotterbeck, Kurt Walsen

# Tarea 1

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno
from sklearn.model_selection import train_test_split

# P1 Limpieza de datos

## Parte 1

In [ ]:
# Se carga una de las semanas de la data para inspeccionar:
w13_all = pd.read_csv('./data/raw/w{}/metrocuadrado_all_w{}.csv'.format(13,13))
w13_fur= pd.read_csv('./data/raw/w{}/metrocuadrado_furnished_w{}.csv'.format(13,13))

In [ ]:
# Inspeccionamos sus columnas, y notamos que son las mismas
print(w13_all.columns)
print(w13_fur.columns)
w13_all.columns==w13_fur.columns

In [ ]:
'''Realizamos una concatenación con y sin duplicados para verificar la 
existencia de éstos.
'''
df_con=pd.concat([w13_all,w13_fur],ignore_index=True)
df_con.head()

In [ ]:
# Verificamos si existen duplicados
df_con.shape[0]==df_con.drop_duplicates().shape[0] # False, luego existen.

In [ ]:
'''
Buscamos generar una variable categórica que indique si la observación
correspondiente proviene de un archivo que en su nombre contiene `furnished`.
Para ello haremos uso de la función merge.

Notamos que la columna que sirve de identificador es `url` pues debiese 
ser único para cada propiedad en arriendo. Además podemos utilizarlo como llave
para hacer el merge de los dataframes y ver qué ocurre con al columna
`furnished`.

De esta manera identificamos a archivos de texto `furnished` que no estén
contenidos en archivos con texto `all` mediante la inspección de los valores
right_only en la columna 'furnished'.
'''
w13_all = w13_all.drop_duplicates()
w13_fur = w13_fur.drop_duplicates()


df_mer = pd.merge(left=w13_all,right=w13_fur,on='url',how='outer',indicator='furnished')
df_mer

In [ ]:
'''Para w13 podemos notar 5 de éstos casos. Pero hay que verlo para el caso
general.
'''
fur=df_mer.drop_duplicates()[['url','furnished']]
fur[fur['furnished']=='right_only']

In [ ]:
'''Ahora, agregamos al dataframe de concatenación la columna con la variable
categórica furnished, mediante el uso de merge nuevamente.
'''
df_con = df_con.drop_duplicates()
pd.merge(df_con,fur,on='url').drop_duplicates()

In [ ]:
'''Se compara con el DataFrame sin la columna agregada, se observa que poseen
la misma cantidad de registros.
'''
df_con

In [ ]:
w13_fur.info()

In [ ]:
w13_all.info()

In [ ]:
# Para unificar formatos, pasamos las columnas 'n_rooms' y 'n_bath' de _fur a string.
w13_fur.n_rooms=w13_fur.n_rooms.astype(str)
w13_fur.n_bath=w13_fur.n_bath.astype(str)
w13_fur.info()

In [ ]:
'''Ahora, realizamos el proceso general para todas las semanas disponibles
en la data.
'''
# Se concatenan todos los _all
df_all = pd.concat([pd.read_csv('./data/raw/w{}/metrocuadrado_all_w{}.csv'.format(i,i)) for i in range(13,18)])
print('Número de registros en df_all: '+str(len(df_all)))
df_all.drop_duplicates(inplace=True)
print('Número de registros en df_all sin duplicados: '+str(len(df_all)))

# Se concatenan todos los _fur
df_fur = pd.concat([pd.read_csv('./data/raw/w{}/metrocuadrado_furnished_w{}.csv'.format(i,i)) for i in range(13,18)])
print('Número de registros en df_fur: '+str(len(df_fur)))
df_fur.drop_duplicates(inplace=True)
df_fur.reset_index()
print('Número de registros en df_fur sin duplicados: '+str(len(df_fur)))

furnished_only = df_fur.query("url not in @df_all.url")
print("Observaciones de archivos con texto 'furnished' que no estén\n contenidos en archivos con texto 'all': ",
      len(furnished_only))
# Se agregan los que estan solo en furnished
df = pd.concat([df_all, df_fur.query("url not in @df_all.url")],ignore_index=True)

# Se crea columna dummy y se le asigna valor 1 a los que se encontraban en archivos furnished
from_furnished = df.query("url in @df_fur.url").index
df.loc[:, 'furnished'] = np.zeros(len(df), dtype=int)
df.loc[from_furnished, 'furnished'] = 1

In [ ]:
df

## Parte 2

In [ ]:
'''Se procede a limpiar parte de la data, esto incluye cambiar el formato
de ciertas variables de interés. En particular:

- 'price' : Se decidió expresar la columna de precios como un número flotante,
            para evitar posibles futuras complicaciones en la inferencia 
            estadística.
            
- 'n_rooms' : Se decidió expresar la columna de número de habitaciones como 
              str, pues debido a la forma de los valores en esta columna
              es posible categorizarlas en base a la cantidad de baños 
              que registra, de esta manera los datos del tipo '5+' serán
              parte de una única categoría.
- 'n_bath' : Análogo a 'n_rooms'.
              
'''
# Se actualiza la columna 'price'
df['price'] = df['price'].str.replace('.','').str.strip('$').map(float)

# Se genera un diccionario de reemplazos
repl_dic = {1.0:'1',2.0:'2',3.0:'3',4.0:'4',5.0:'5'}

# Se actualizan las columnas 'n_rooms' y 'n_bath'
df['n_rooms'] = df['n_rooms'].replace(repl_dic)
df['n_bath'] = df['n_bath'].replace(repl_dic)


'''Se procede a separar el contenido de la columna 
'property_type|rent_type|location' en tres nuevas columnas 'property_type', 
'rent_type' y 'location'.
'''
# Preparamos la columna a tratar
newcols = df['property_type|rent_type|location'].str.split(pat= ',',expand=True)

# Se generan las nuevas columnas
df['location'] = newcols[1]
df[['property_type','rent_type']]=newcols[0].str.lower().str.split(pat=' ',n=1,expand=True)
df.loc[:,'rent_type'] = df['rent_type'].str.lstrip('en ')


# Fijamos las columnas de df en orden
cols = ['property_type', 'rent_type', 'location', 'price', 'n_rooms',
        'n_bath', 'surface', 'details', 'url', 'metrocuadrado_index',
        'furnished']
# Actualizamos df
df = df[cols]

## Parte 3

In [ ]:
'''Se procede a agregar las columnas 'price_per_m2' y 'n_garajes',donde:

- 'price_per_m2' : Representa el precio por metro cuadrado.

- 'n_garajes' : Representa el número de garajes.

'''
# Modificamos los valores de la columna 'surface' para que representen el
# valor en metros cuadrados como número flotante.

df['surface'] = df['surface'].replace('m2', '', regex=True).map(float)

# Generamos la nueva columna 'price_per_m2' 
df['price_per_m2'] = df['price'] / df['surface']



'''Para 'n_garajes' fue un proceso más complejo. Notamos que el str 
asociado a cada elemento en 'url' es de la siguiente forma:
'''
print(df['url'][0])

'''Sin embargo, este no es el caso de todas las url. En particular:
'''
print(df['url'][3])

In [ ]:
'''Luego, una manera intuitiva de obtener el número de garajes es separar el
str con el separador '-garajes' lo cual generará una lista de dos strings, 
luego podemos acceder al primer string de la lista y extraer el último elemento,
recuperando así el número de garajes.
Lo cual nos entrega los siguientes valores para 'n_garajes':
'''
df['n_garajes'] = df['url'].map(lambda string: string.split(sep='-garajes')[0][-1])

df['n_garajes'].unique()

In [ ]:
#ojo acá
print(df['url'][3])
df['n_garajes'][3]

In [ ]:
'''Notamos que aparece un elemento de la forma '+', luego al igual que
'n_rooms' y 'n_bath', existe una categoria que representa una cantidad
superior a cierto valor, la intuición nos dice que tal categoria
corresponde a '9+'.
En búsqueda de generalizar la transformación(y así incluir éste tipo de
categorías) se uso una lambda function distinta, dando como resultado 
lo siguiente:
'''
df['n_garajes'] = df['url'].map(lambda string: string.split(sep='-garajes')[0][-1] if string.split(sep='-garajes')[0][-1] !='+' else string.split(sep='-garajes')[0][-2:])
df['n_garajes'].unique()

In [ ]:
'''Ésto nos muestra que la única categoria de la forma antes descrita 
corresponde solamente a '4+', algo confuso pues ya existen las categorías 5:9.
Notamos que la cantidad de datos que corresponden a éste tipo de categoría 
son 9. 
'''
print(len(df['n_garajes'][df['n_garajes']=='4+']))

In [ ]:
'''Veamos ahora cúantos corresponden a las categorías >4, notamos 
que corresponden a 1010(bastante mayor en comparación a '4+').
'''
print(len(df['n_garajes'][df['n_garajes']>'4']))

In [ ]:
'''Sin embargo, notamos que la cantidad de datos que pueden representarse
en la categoría '4+' corresponden a 1010(notar que al computar '4+'>'4'
nos entrega True), una cifra bastante menor a los 16299 datos totales de
los cuales disponemos, más precisamente, tenemos que las variables que superan
la categoría '4' corresponden en promedio a 1010/6=168.3 por categoría, mientras 
que las variables en categoría '4' o inferior corresponden en promedio a 
(16299-1010)/5=3057.8 por categoría. En base a ésto, es correcto aseverar que
podemos agrupar las variables con categoria superior a '4' en una única 
categoría '4+', sin perder variabilidad en el feature 'n_garajes'.
Por lo tanto, generamos un diccionario que mapee todos las categorias
resultantes superiores a '4' en una única categoría '4+'.
'''
# Se genera diccionario de mapeos
map_dict={'+':'4+', '5':'4+', '6':'4+', '7':'4+', '8':'4+', '9':'4+'}

# Se genera la correcta columna 'n_garajes'
df['n_garajes'] = df['url'].map(lambda string: string[
    string.find('-garajes')-1] if string.find('-garajes') > 0 else np.nan )
df['n_garajes'] = df['n_garajes'].replace(map_dict)
df['n_garajes'].unique()

In [ ]:
# Fijamos el orden de las columnas en df.
cols = ['property_type', 'rent_type', 'location', 'price','price_per_m2', 
        'n_rooms', 'n_bath', 'n_garajes','surface', 'details', 'url', 
        'metrocuadrado_index', 'furnished']

# Reordenamos df.
df = df[cols]

In [ ]:
df

## Parte 4

In [ ]:
'''Queremos categorizar los productos disponibles en la data según
el tipo de inmueble al que corresponde y la cantidad de m2 de 
superficie que poseen, para ello haremos uso de 8 categorias:

1 : 'rent_type' = 'casa' , 80 <= 'surface' < 120
2 : 'rent_type' = 'casa' , 120 <= 'surface' < 180
3 : 'rent_type' = 'casa' , 180 <= 'surface' < 240
4 : 'rent_type' = 'casa' , 240 <= 'surface' < 360
5 : 'rent_type' = 'casa' , 360 <= 'surface' < 460
6 : 'rent_type' = 'apartamento' , 40 <= 'surface' < 60
7 : 'rent_type' = 'apartamento' , 60 <= 'surface' < 80
8 : 'rent_type' = 'apartamento' , 80 <= 'surface' < 120

'''

# Se crea columna para ser rellenada a posteriori
df['product_type'] = np.repeat([np.nan], len(df))

# Se rellenan los tipos para las casas
cotas_casas = [(80, 120), (120, 180), (180, 240), (240, 360), 
               (360,460)]

for i in range(len(cotas_casas)):
    q = "(property_type == 'casa') & ({0} < surface <= {1})".format(*cotas_casas[i])
    idx = df.query(q).index
    df.loc[idx, 'product_type'] = str(i+1)

# Se rellenan los tipos para apartamentos    
cotas_apartamentos = [(40, 60), (60, 80), (80, 120)]

for i in range(len(cotas_apartamentos)):
    q = "(property_type == 'apartamento') & ({0} < surface <= {1})".format(*cotas_apartamentos[i])
    idx = df.query(q).index
    df.loc[idx, 'product_type'] = str(i+(len(cotas_casas)+1))
    
print('Cantidad de productos no clasificados: '
      ,len(df[df['product_type'].isna()]))
print('Categorías: ',df['product_type'].unique())

In [ ]:
df

## Parte 5

In [ ]:
'''Queremos generar una nueva columna que indique el barrio, a partir de location
Notamos que todas las location poseen la estructuca '{barrio} Bogotá D.C.'
'''
df.location.unique()[:20]

In [ ]:
'''Verificamos si hay locations que no contengan esta keyword, 
notamos que todos la tienen.
'''
df[df['location'].map(lambda string: not('Bogotá D.C.' in string))]

In [ ]:
'''Procedemos a generar la columna 'barrio', de manera similar a como 
obtuvimos la columna 'n_garajes'
'''
df.loc[df.index, 'barrio'] = df['location'].map(lambda string: 
                                                string.split(sep='Bogotá')[0].strip(' ').lower())
print('Cantidad única de barrios disponibles: ',len(df['barrio'].unique()))
df[['location','barrio']].head()

In [ ]:
# Se examina barrio-upz.csv para determinar cómo hacer el merge
upz=pd.read_csv('./data/asignacion_upz/barrio-upz.csv')
upz.head(10)

In [ ]:
print('Cantidad única de UPlNombre disponibles: ',len(upz.UPlNombre.unique()))
upz.UPlNombre.unique()[:20]

In [ ]:
print('Cantidad única de pro_location disponibles: ',len(upz.pro_location.unique()))
upz.pro_location.unique()[:20]

In [ ]:
'''Notamos que 'pro_location' presenta una gama más amplia de barrios sobre los
cuales poder cruzar la data. En base a esto, haremos un cruce entre df y
asignacion_upz.
'''
# Se realiza un merge 'outer' para determinar cuántos no tienen código upz
df_merged = pd.merge(df,upz,left_on='barrio',right_on='pro_location',
                     how='outer',indicator='ind')

barrios_upz = df_merged[df_merged['ind']=='right_only']['UPlNombre'].nunique()
obs_sin = len(df_merged[df_merged['ind']=='left_only'])
print('Barrios con código UPZ que no están en df: '+str(barrios_upz))
print('Observaciones sin código UPZ: '+str(obs_sin))
print('Barrios sin código UPZ: '+str(df_merged[df_merged['ind']=='left_only'].barrio.nunique()))
porc = int(100*len(df_merged[df_merged['ind']=='both'])/len(df))
print('El {}% de las observaciones tienen código UPZ'.format(porc))

'''Notamos que existen 13 barrios de la data upz los cuales no
aparecen en nuestro df, ésto se quizás a que no existen publicaciones de 
arriendo/venta de propiedades pertenecientes a tales barrios.
Además, se observa que 1946 registros pertenecen a barrios los cuales no se 
les puede adjuntar un código UPZ. De éstos notamos que la cantidad única de 
barrios corresponde a 176. Además, un 88% de las observaciones en df poseen
un código UPZ.
'''

# Se define el nuevo df
#df = df_merged[df_merged['ind']=='both'].drop(columns=['UPlTipo','UPlNombre','ind','pro_location'])
df = df_merged[df_merged['ind']!='right_only'].drop(columns=['UPlTipo','UPlNombre','ind','pro_location'])

df.reset_index(drop=True)

# Fijamos las columnas en df.
cols = ['product_type','property_type', 'rent_type', 'location','barrio','UPlCodigo',
        'UPlArea', 'price','price_per_m2','surface', 'n_rooms', 'n_bath',
        'n_garajes', 'details', 'url', 'metrocuadrado_index', 'furnished']

df = df[cols]

In [ ]:
#
df

## Parte 6

In [ ]:
# Cargamos la data asociada a estadisticas_poblacion.csv
stats_pob=pd.read_csv('./data/estadisticas_upz/estadisticas_poblacion.csv')
stats_pob.head()

In [ ]:
# Estructura columna de UPlCodigo para la data stats_pob, a priori
# no se observas complicaciones en el formato.
stats_pob.upz.unique()

In [ ]:
# Rescatamos solo las columnas relevantes
stats_pob.drop(columns=['Unnamed: 0','nomupz'],inplace=True)
df_merged = pd.merge(df,stats_pob,left_on='UPlCodigo',right_on='upz',how='left')
df_merged

In [ ]:
# Cargamos la data asociada a indice_inseguridad.csv
ind_inseg=pd.read_csv('./data/estadisticas_upz/indice_inseguridad.csv')
ind_inseg.head()

In [ ]:
# Notamos que hay códigos '1', '3', '4' y '5', pero también notamos
# que éstos UPZ{} no están en df ni coinciden con otros, por lo que 
# los ignoramos.
print('Códigos UPZ en ind_inseg: ',ind_inseg.UPlCodigo.unique())
ind_inseg.drop(columns=['Unnamed: 0','UPlNombre2'],inplace=True)
df_merged = pd.merge(df_merged,ind_inseg,on='UPlCodigo',how='left')

In [ ]:
df_merged

In [ ]:
# Cargamos la data asociada a porcentaje_areas_verdes.csv
perc_areas_verdes = pd.read_csv('./data/estadisticas_upz/porcentaje_areas_verdes.csv')
perc_areas_verdes.head()

In [ ]:
# Estructura columna de UPlCodigo para la data perc_areas_verdes,
# notamos que contiene solo floats, sin el prefijo 'UPZ'
print('Previo a la transformación:\n',perc_areas_verdes['cod_upz'].unique())
# Se tranforma x en str de la forma 'UPZ'+str(int(x))
perc_areas_verdes.loc[:,'cod_upz']=perc_areas_verdes['cod_upz'].map(lambda x: 'UPZ'+str(int(x)))
print('Luego de la transformación:\n',perc_areas_verdes['cod_upz'].unique())

# Se dropean columnas innecesarias y se hace un último merge
perc_areas_verdes.drop(columns=['Unnamed: 0','upz'],inplace=True)
df_merged = pd.merge(df_merged,perc_areas_verdes,left_on='UPlCodigo',right_on='cod_upz',how='left')

In [ ]:
df_merged

In [ ]:
# Hay 3 columnas que dicen lo mismo: 'UPlCodigo','upz','cod_upz'. Nos quedamos con la segunda por simplicidad
df=df_merged.drop(columns=['UPlCodigo','cod_upz'])

In [ ]:
# Generamos la nueva columna con la densidad de población por UPZ
df['densidad_poblacion']=df['personas']/df['UPlArea']

In [ ]:
# Botamos los últimos duplicados
df.drop_duplicates(inplace=True)
df.reset_index(drop=True)
df

In [ ]:
# Se guarda una copia de la data procesada hasta ahora
#df_raw = df.copy()

# P2

## Parte 1

In [ ]:
# Creamos una función que fije cómo queremos que se vean los plots.
def estilo():
    sns.set(style='darkgrid')
    plt.rcParams['figure.figsize'] = (18, 18)
estilo()

## Parte 2

In [ ]:
'''Buscamos hacer un perfilamiento de las variables disponibles en la data
a partir de la parte anterior.
'''
print(df.shape)
print(df.info())

In [ ]:
df.tail(10)

In [ ]:
# Tipos de variables que queremos
names = ['numeric','categorical','miscelaneous']

# Fijamos las variables numéricas
numeric = ['UPlArea','price','surface','metrocuadrado_index',
           'personas', 'trabajoinf_ninos_5_17_anos_perc',
           'trabajoinfampliado_ninos_5_17_anos_perc',
           'jovenes_14_24_anos_nini_perc','indice_envegecimiento',
           'jefe_mujer_perc','adultos_mayores_pobres_perc','indice_inseguridad',
           'areas_verdes_perc','densidad_poblacion','price_per_m2']

# Fijamos las variables miscelaneas, recordemos que el barrio
# puede identificarse con un código upz
miscelaneous= ['location','barrio','url','details']

# Se crea una lista con las variables categoricas
categorical = list((set(df.columns) - set(numeric)) - set(miscelaneous))
categorical

In [ ]:
# Generamos el mapeo a multi-índices
mapping = [('numeric', col) for col in numeric]
mapping.extend([('categorical', col) for col in categorical])
mapping.extend([('miscelaneous', col) for col in miscelaneous])

In [ ]:
# Re-indexamos la data
df = df.reindex(columns=numeric + categorical + miscelaneous)
df.columns = pd.MultiIndex.from_tuples(mapping)
df

In [ ]:
'''Variables numéricas

Previo a graficar las variables numéricas, debemos notar que existen
registros en los cuales 'surface' es 0( y por ende 'price_per_m2' es inf)
'''
zero_surf=df[df[('numeric','surface')] == 0][[('numeric','price'),('numeric','surface'),
                                   ('numeric','price_per_m2')]]
nonzero_surf=df[df[('numeric','surface')] > 0][[('numeric','price'),('numeric','surface'),
                                   ('numeric','price_per_m2')]]
print("Total de registros: ",len(df))
print("Registros con 'surface'=0 : ",len(zero_surf))
print("Registros con 'surface'>0 : ",len(nonzero_surf))
zero_surf

In [ ]:
'''Luego podemos considerar estos datos como faltantes, pues en la
práctica no existen propiedades de 0 m2 con tales precios.
'''
df.loc[:,[('numeric','surface')]] = df[[('numeric','surface')]].replace(float(0),np.nan)
df.loc[:,[('numeric','price_per_m2')]] = df[[('numeric','price_per_m2')]].replace(np.inf,np.nan)

In [ ]:
# Verificamos que el cambio se realizó correctamente
df[df[('numeric','surface')].isnull()][[('numeric','price'),('numeric','surface'),
                                   ('numeric','price_per_m2')]]

In [ ]:
'''Guardaremos una copia de df para poder hacer modificaciones en
el camino y de esta manera entender mejor las visualizaciones. 
Más adelante veremos que tales modificaciones resultan útiles a la hora
de poder obtener una mejor identificación entre las variables.
'''
df_copy = df.copy()

In [ ]:
'''Se procede a graficar las distribuciones de las variables numéricas
'''
# Grilla de subplots
fig, ax = plt.subplots(nrows=4, ncols=4)#, figsize=[17, 17])

# Se remueven el ultimo plot
list(map(lambda a : a.remove(), ax[-1,-1:]))

# Se ajusta el espaciado exterior de la figura
fig.tight_layout()

# Se define un titulo y su ubicacion
fig.suptitle('Distribuciones Univariadas Numéricas',
             fontsize=20,
             x=0.5,
             y=1.05)
'''
Se recorre cada axis, para cada columna del dataframe, se genera un grafico 
distinto en funcion del tipo de dato.

'''
for axis, col in zip(ax.flatten(), numeric):
    try :
        # Graficos para datos numericos
        sns.distplot(df[('numeric', col)].dropna(), ax=axis, rug=True)
               
    except RuntimeError:
        sns.distplot(df[('numeric', col)].dropna(), ax=axis, rug=True, kde=False)
    
    axis.set_xlabel(col, fontsize=15)

# Se ajusta el espaciado interno entre subplots
w, h = (.4, .4)
plt.subplots_adjust(wspace=w, hspace=h)

In [ ]:
'''Notamos en 'price' y 'price per_m2' que hay una diferencia muy brusca en la
densidad de los datos, donde la tendencia es clara a precios mas moderados en
comparacion.
Dado que 'price_per_m2' es nuestra variable respuesta, nos interesa que esta
presente una distribución que sea lo suficientemente suave para 
poder realizar futuras transformaciones en nuestro futuro modelo regresor.
Notamos que considerando datos con valor de 'price_per_m2' < 100.000, se 
obtiene el siguiente dataframe.
'''
df[df[('numeric','price_per_m2')]<=10**5]

In [ ]:
'''Tenemos una diferencia de 100 datos con la data original, es decir,
estamos ignorando los 100 valores donde 'price_per_m2' > 100.000. Veamos 
ahora como se ve su distribución.
'''
sns.distplot(df[df[('numeric','price_per_m2')] <= 10**5][('numeric','price_per_m2')])

In [ ]:
'''Notamos una clara mejora con respecto al escenario con toda la data.
Actualizamos df y vemos ahora como distribuyen todas las variables numericas
asociadas.
'''
df = df[df[('numeric','price_per_m2')] <= 10**5]
df

In [ ]:
'''Se procede a graficar las distribuciones de las variables numéricas
luego del ajuste
'''
# Grilla de subplots
fig, ax = plt.subplots(nrows=4, ncols=4)#, figsize=[17, 17])

# Se remueven el ultimo plot
list(map(lambda a : a.remove(), ax[-1,-1:]))

# Se ajusta el espaciado exterior de la figura
fig.tight_layout()

# Se define un titulo y su ubicacion
fig.suptitle('Distribuciones Univariadas Numéricas',
             fontsize=20,
             x=0.5,
             y=1.05)
'''
Se recorre cada axis, para cada columna del dataframe, se genera un grafico 
distinto en funcion del tipo de dato.

'''
for axis, col in zip(ax.flatten(), numeric):
    try :
        # Graficos para datos numericos
        sns.distplot(df[('numeric', col)].dropna(), ax=axis, rug=True)
               
    except RuntimeError:
        sns.distplot(df[('numeric', col)].dropna(), ax=axis, rug=True, kde=False)
    
    axis.set_xlabel(col, fontsize=15)

# Se ajusta el espaciado interno entre subplots
w, h = (.4, .4)
plt.subplots_adjust(wspace=w, hspace=h)

In [ ]:
'''Notamos como la distribución de la data es más clara al trabajar sin los casos
donde price_per_m2 es demasiado grande. Notamos en surface que existe un dato que se aleja
mucho de donde se concentra la data, y su valor es al menos superior a 1000, donde los 
demas valores se concentran por debajo de 1000. Buscamos tal valor
'''
df[('numeric','surface')].nlargest(2)

In [ ]:
'''Veamos qué ocurre al quitar tal valor del dataframe, notamos una
clara mejora en la visualización.
'''
sns.distplot(df[df[('numeric','surface')]< 10**3][('numeric','surface')])

In [ ]:
# Se actualiza df
df = df[df[('numeric','surface')]< 10**3]
df.reset_index(inplace=True, drop=True)

In [ ]:
# Grilla de subplots
fig, ax = plt.subplots(nrows=4, ncols=4)#, figsize=[17, 17])

# Se remueven el ultimo plot
list(map(lambda a : a.remove(), ax[-1,-1:]))

# Se ajusta el espaciado exterior de la figura
fig.tight_layout()

# Se define un titulo y su ubicacion
fig.suptitle('Distribuciones Univariadas Numéricas',
             fontsize=20,
             x=0.5,
             y=1.05)
'''
Se recorre cada axis, para cada columna del dataframe, se genera un grafico 
distinto en funcion del tipo de dato.

'''
for axis, col in zip(ax.flatten(), numeric):
    try :
        # Graficos para datos numericos
        sns.distplot(df[('numeric', col)].dropna(), ax=axis, rug=True)
               
    except RuntimeError:
        sns.distplot(df[('numeric', col)].dropna(), ax=axis, rug=True, kde=False)
    
    axis.set_xlabel(col, fontsize=15)

# Se ajusta el espaciado interno entre subplots
w, h = (.4, .4)
plt.subplots_adjust(wspace=w, hspace=h)

In [ ]:
'''Queremos ver como se comporta la variable 'price_per_m2' en respuesta
a algunas variables numéricas.
'''
def scatter_dists(col, df=df, h=.3, w=.1, fontdict={'fontsize': 20}, reg=True):
    ''' Recibe una columna numerica y genera una visualizacion comparativa.
    
    Genera una figura por sobre el dataframe (por defecto), recibe 
    parametros extra como el espaciado entre subfigura.
    
    Args:
    ----------
    
    col: String
         El nombre de la columna numerica a visualizar
    
    h,w: float
        Espaciado entre subplot h -> vertical, w -> horizontal
    
    fontdict: dict
             Permite configurar las fuentes de los subplots
    reg: bool
         Permite graficar una regresion lineal sobre los datos (if True)
        
    Returns: None
        Se muestra una figura en pantalla    
    
    '''

    # Estrucutra de figura y axes
    fig, ax = plt.subplots(2, 1, figsize=[12, 13])

    # violin plot --> equivalente a catplot(kind = 'violin')

    if reg:
        sns.regplot(x=df[('numeric', col)],
                    y=df[('numeric', 'price_per_m2')],
                    ax=ax[0])
        ax[0].set_title('Regplot plot {} vs price_per_m2'.format(col), fontdict)
    else:
        sns.scatterplot(('numeric', col),
                        y=('numeric', 'price_per_m2'),
                        data=df,
                        ax=ax[0])
        ax[0].set_title('Scatter plot {} vs price_per_m2'.format(col), fontdict)

    
    # Distribucion univariada
    sns.distplot(df[('numeric', col)].dropna(), ax=ax[1])

    ax[0].set_xlabel(col, fontdict)
    ax[1].set_xlabel(col, fontdict)

    ax[0].set_ylabel('price_per_m2', fontdict)
    ax[1].set_title('Frecuencias {}'.format(col), fontdict)

    plt.subplots_adjust(wspace=w, hspace=h)


In [ ]:
'''Notamos que la variable 'metrocuadrado_index', presenta una buena 
distribución en la data y presenta un comportamiento lineal con ruido, puede que esta 
variable sea de interés a la hora de regresionar 'price_per_m2'.
'''
scatter_dists('metrocuadrado_index')

In [ ]:
'''Notamos que las variables 'price' y 'surface' presentan un comportamiento
lineal con respecto a 'price_per_m2', pero esto se debe a como se generó la
columna 'price_per_m2', luego la relación existente entre éstas variables
fue impuesta y no presenta un caso de interés.
'''
scatter_dists('price')

In [ ]:
scatter_dists('surface')

In [ ]:
'''Ruido, 'price_per_m2' no presenta ninguna respuesta clara ante
esta variable. Su significancia deberá ser evaluada más adelante.

Resultado análogo entre
'UPlArea'
'trabajoinf_ninos_5_17_anos_perc'
'trabajoinfampliado_ninos_5_17_anos_perc'
'jovenes_14_24_anos_nini_perc'
'indice_envegecimiento'
'jefe_mujer_perc'
'adultos_mayores_pobres_perc'
'indice_inseguridad'
'areas_verdes_perc'
'densidad_poblacion'
'''
scatter_dists('UPlArea')

In [ ]:
'''Variables categóricas
Procedemos a analizar las variables categóricas
'''
print(len(categorical))
print(categorical)

In [ ]:
'''Se procede a graficar los histogramas de las variables categóricas
'''
# Grilla de subplots
fig, ax = plt.subplots(nrows=3, ncols=3)

# Se remueven el ultimo plot
list(map(lambda a : a.remove(), ax[-1,-1:]))

# Se ajusta el espaciado exterior de la figura
fig.tight_layout()
# Se define un titulo y su ubicacion
fig.suptitle('Distribuciones Univariadas Categóricas',
             fontsize=20,
             x=0.5,
             y=1.05)
'''
Se recorre cada axis, para cada columna del dataframe, se genera un grafico 
distinto en funcion del tipo de dato.

'''
for axis, col in zip(ax.flatten(), categorical):
    # Graficos para datos tipos str
    sns.countplot(df[('categorical',col)], ax=axis)
    axis.set_axis_off()
    axis.set_title(col, fontsize=15)
  
    
# Se ajusta el espaciado interno entre subplots
h, w = (.4, .1)
plt.subplots_adjust(wspace=w, hspace=h)

In [ ]:
'''Estudiemos la variabilidad de las variables categóricas ''
'''
# Función para generar gráficos
def categoricalplot(df,col,log=False):
    # Sirve para fija el tamaño de las etiquetas del plot
    fontdict = {'fontsize':20}

    # Estrucutra de figura y axes
    fig, ax = plt.subplots(2,1,figsize=[12,13])
    
    # violin plot --> equivalente a catplot(kind = 'violin')

    if log:
        sns.violinplot(('categorical', col),
                    y=('numeric', 'price_per_m2'),
                    data=df,
                    kind='violin',
                    ax=ax[0]).set_yscale('log')
    
    else:
        sns.violinplot(('categorical', col),
                    y=('numeric', 'price_per_m2'),
                    data=df,
                    kind='violin',
                    ax=ax[0])
    
    sns.countplot(df[('categorical',col)], ax=ax[1])

    ax[0].set_xlabel(col, fontdict)
    ax[1].set_xlabel(col, fontdict)

    ax[0].set_ylabel('precio_per_m2', fontdict)
    ax[0].set_title('Violin plot {} vs price_per_m2'.format(col), fontdict)
    ax[1].set_title('Frecuencias {}'.format(col), fontdict)

    h, w = (.3, .1)
    plt.subplots_adjust(wspace=w, hspace=h)

In [ ]:
'''upz
Observamos que la variable categoriza upz presenta un histograma a la vista
confuso(demasiadas categorias), por lo que más adelante buscaremos una mejor
manera de agrupar éstas categorías.
'''
categoricalplot(df,'upz')

In [ ]:
'''product_type
Notamos que para los 'product_type' 1-5 (casas), presentan una
concentración en valores levemente más bajos que los 6-8 (apartamentos), además
la variabilidad del valor de precio dado si es casa o apartamento presentan una
distribución similar. Por ende product_type podría corresponder a una variable
de interés a la hora de definir 'price_per_m2'.
'''
categoricalplot(df,'product_type')

In [ ]:
'''property_type
Notamos que los valores de 'precio_per_m2' para la categoría 'casa' se concentran
en su mayoría en un valor menor que la categoría apartamento. Ésto se condice
con el gráfico anterior y se debe en parte a cómo definimos la variable
'product_type', luego puede existir cierta correlación entre éstas dos variables,
lo cual puede corroborarse mediante un test estadístico.
'''
categoricalplot(df,'property_type')

In [ ]:
'''rent_type
Notamos que no difieren en mediana y sus distribuciones en 'price_per_m2'
se comportan de manera similar, por ende no existe una manera de poder
identificar una de las categorías en base al valor de 'price_per_m2'.
Por lo tanto, 'rent_type' corresponde a una variable candidata a no ser 
considerada en el modelo final.
'''
categoricalplot(df,'rent_type')

In [ ]:
'''furnished
Si bien no se nota una diferencia en los valores donde más se cocentra
cada categoría, el hecho de la clara diferencia en la distribución de
la variable nos hace dudar sobre la efectividad en describir la variable
'price_per_m2', en caso de ser incluída en el modelo. Por lo tanto,
más adelante se verá si incluir o no esta variable en el modelo final.
'''
categoricalplot(df,'furnished')

In [ ]:
'''n_rooms
Se notan claras diferencias en las medias del valor 'price_per_m2'
entre categorias, ademas de una distribución normalmente aproximable
en los valores en esta variable. Por lo tanto corresponde a un candidato
sólido a ser incluido en el modelo final.
'''
categoricalplot(df,'n_rooms')

In [ ]:
'''n_bath
Si bien la diferencia de medias en este caso tambien existe pero en menor
medida, la forma variabilidad en su distribución puede ser información 
valiosa para la descripción de 'price_per_m2', por lo tanto a priori es 
una variable interesante a considerar.
'''
categoricalplot(df,'n_bath')

In [ ]:
'''n_garajes
Tenemos un caso similar a 'n_rooms', donde aquí la diferencia de medias
es menos clara, y debido a que los violines son más achatados nos hace
dudar sobre la correcta descripción de 'price_per_m2' por medio de esta
variable. Veremos mediante un test one-way-ANOVA si existe una diferencia
significativa entre grupos.
'''
categoricalplot(df,'n_garajes')

In [ ]:
'''Se genera una función auxiliar para indexar las columnas en base
a su tipo, extraído de la clase 9 del curso.
'''
def indexer(cols, t_c = df.columns):
    '''Genera columnas multinivel a partir de nombres de columna planos.'''
    
    set_to_tuple = set(*[cols])

    tuples = [
        i for i in t_c if set_to_tuple.intersection(set(i))
    ]
    
    return tuples

In [ ]:
idx = indexer(['price_per_m2','n_garajes'])
grouped = df[idx].groupby(idx[1])
total_groups = grouped.groups.keys()
groups = [grouped.get_group(i) for i in total_groups]
groups[0]

In [ ]:
'''Se utiliza una función auxiliar para limpiar el formato de cada grupo,
extraído de la clase 9 del curso.
'''
def group_cleaner(group, col, d_f=df):
    ''' Limpia un grupo.
    Reconoce la categoria del grupo, en la posicion [:,1], 
    guarda ese nombre y elimina la columna de categoria, 
    posteriormente renombra la columna.
    
    Args:
    ----------
    
    group: pandas Groupby object
          Recibe una agrupacion para categorias
          
    Returns:
    ----------
        pandas Groupby object
        Entrega el grupo ordenado.
    '''
    group_0 = group.copy()
    name = group_0.iloc[0,1]
    group_0.drop(indexer([col], t_c = d_f.columns), axis=1, inplace=True)
    group_0.columns  = ('cat_{}'.format(name),)
    
    return group_0

In [ ]:
'''Se procede a limpiar el formato de cada grupo y se realiza el test
'''
groups_to_test = [group_cleaner(g, 'n_garajes') for g in groups]

from scipy.stats import f_oneway

F,p = f_oneway(*groups_to_test)

print('Estadistico F:',F)
print('p valor :', p)

In [ ]:
'''Con esto, rechazamos la nula: 'No hay diferencia significativa entre
grupos'. Por lo tanto, 'n_garajes' corresponde a una variable de interés
a analizar.
'''
alpha = 0.05
p <= alpha

## Parte 3

In [ ]:
'''Para ver las variables faltantes, recuperamos df_copy previo a las
modificaciones hechas en P2.2
'''
df_copy.isnull().sum()

In [ ]:
'''Vemos un esquema general de los valores faltantes en de la data,
se puede observar un comportamiento similar en la ausencia de los 
datos asociados a las estadísticas incluidas mediante un cruce con los
códigos upz. Ésto claramente es debido a que al existir barrios donde no 
fue posible obtener identificación mediante el código upz, no fue posible
cruzar las estadísticas en la sección P1.6, por lo tanto la ausencia
de las estadísticas se refleja en la ausencia de upz. El tratamiento
para estos datos faltantes será eliminarlos, pues debido a no poder
recuperar el upz, no podremos recuperar de manera consistente las
estadísticas.
'''
fig, ax = plt.subplots(figsize = [15, 10])
msno.matrix(df_copy,ax = ax, sparkline=False)

In [ ]:
'''Esquema ordenado según cantidad de datos faltantes
Observamos que las columnas con más datos faltantes correspondes a
`product_type` y 'n_garajes'
-'product_type' se debe a como la definimos en 
la sección P1.4, luego depende de parámetros visibles en la data
('property_type','rent_type','surface'). Nuestro tratamiento para ésta 
variable será simplemente eliminarlas.
-'n_garajes' se debe a la ausencia de la keyword '-garajes' en url. Por
lo tanto, la ausencia de ésta data se puede inferir a partir de la 
variable 'url', variable sobre la cual fue construida esta columna.

Respecto a las columnas 'n_bath','details','n_rooms','price','surface'
no es posible determinar un patron claro, más aun cuando la ausencia
de las variables en las ultimas 3 mencionadas son pocas (menos de 33).
Creemos entonces que ésta información es perdida completamente al azar,
pues depende de algo que no estamos viendo reflejado en la data(mal 
ingreso de los datos, omisión de informacion por parte del vendedor,etc.)
Ésta información se intentará imputar asignando media mediante agrupaciónes
por 'upz' donde sea posible.
'''
msno.matrix(df_copy[list(df.isnull().sum().nlargest(19).index)], sparkline=True)

In [ ]:
'''Mediante un mapa de calor podemos identificar la correlación
entre los valores faltantes, para determinar si existe algún tipo
de dependencia en la ausencia de éstos datos.
En efecto, notamos como las variables asociadas a las estadísticas
incluidas en la sección P1.6 presentan correlación 1 entre ellas y 
con upz, confirmando entonces la clara dependencia de la ausencia de 
estos datos en base a la ausencia de upz. 
Además, notamos una trivial correlación de 1 entre surface y 
price_per_m2 debido a que la data faltante(surface=0) genera una
imposibilidad en el cálculo de price_per_m2(sería infinito).
'''
fig, ax = plt.subplots(figsize = [15, 10])
msno.heatmap(df_copy, ax = ax)

In [ ]:
'''A través de un dendograma podemos confirmar la relación de datos 
faltantes, ahora entre grupos. Confirmamos lo mencionado con las 
estadísticas y upz, junto con la relacion 'surface' y 'price_per_m2'.

Se observa cómo 'product_type' no tiene relación alguna en su
ausencia de datos con variables como 'property_type','rent_type',
'surface', sino mas bien en los rangos de valor de éstas. 

Además notamos que la ausencia de datos en 'n_garajes' tambien es ajena
a la ausencia de datos en otras variables expuestas acá.

Concluimos entonces que las variables 'price_per_m2' y 'property_type'
presentan un tipo de mecanismo de pérdida de información del tipo MAR,
pues su ausencia depende de variables que podemos observar su valor:
- 'surface' para 'price_per_m2'
- 'property_type','rent_type','surface' para 'product_type'

Por otro lado, las estadísticas cumplen la hipótesis MNAR, ya que la ausencia de información
en estas variables se explica por la variable ausente 'upz'.
'''
fig, ax = plt.subplots()
msno.dendrogram(df_copy[list(df_copy.isnull().sum().nlargest(19).index)], ax=ax,orientation='top')

In [ ]:
'''Como los nan asociados a upz corresponden a una perdida de informacion del tipo MNAR,
nuestro tratamiento será entonces dropear tales filas.
Como los nan asociados a product_type corresponden a valores que ensucian la variable
respuesta, tambien se procede a dropear tales filas. 
'''
df_copy.dropna(subset=[('categorical', 'upz'),('categorical','product_type')], axis =0,how='any', inplace=True)
df_copy.reset_index(drop=True,inplace=True)
df_raw = df_copy.copy()

In [ ]:
df_copy.isnull().sum()

In [ ]:
# Obtenemos las modas por columna
cols= indexer(['n_rooms','n_bath','n_garajes'])
modes = df_copy[cols].mode(axis=0,dropna=True)
modes

In [ ]:
# Generamos un diccionario de mapeos a utilizar en la imputación
fill_dict={'n_garajes': modes[('categorical','n_garajes')][0],
          'n_rooms': modes[('categorical','n_rooms')][0],
          'n_bath': modes[('categorical','n_bath')][0]
          }
df_copy[cols].isnull().sum()

In [ ]:
# Se procede a imputar
for col in cols:
    df_copy.loc[:,col] = df_copy.fillna(modes[col][0])
df_copy[cols]

In [ ]:
# Notamos que todos las modas fueron imputadas
df_copy[cols].isnull().sum()

## Parte 4

In [ ]:
'''Recordemos como distribuye la variable categorica 'upz'
'''
sns.countplot(df_copy[('categorical','upz')])

In [ ]:
'''Generaremos un dataframe identico al actual pero con los cambios realizados de P2.2
para comparar visualizaciones
'''
df_mod = df_copy.copy()
df_mod =  df_mod[(df_mod[('numeric','price_per_m2')]<=10**5) & (df_mod[('numeric','surface')]<=10**3)] 
df_mod

In [ ]:
'''Buscamos clusterizar los 'upz' en base a su valor en 'price_per_m2'
'''
# Se propone clusterizar la variable upz mediante kmeans
from sklearn.cluster import KMeans

n_clust = 3 # número de clusters a trabajar

# Se inicializan en paralelo 2 clusterizaciones
clusterizer = KMeans(n_clusters=n_clust)
clusterizer_mod = KMeans(n_clusters=n_clust)

In [ ]:
# Clustering mediante la variable de respuesta
# Agrupamos datos por upz y tomamos promedio en la variable de respuesta

# Generamos agrupaciones
grouped_df = df_copy.groupby(by=('categorical','upz')).mean()[('numeric','price_per_m2')]
grouped_df_mod = df_mod.groupby(by=('categorical','upz')).mean()[('numeric','price_per_m2')]

# Guardamos los labels en una serie indexados por el codigo upz
X = clusterizer.fit_predict(grouped_df.to_numpy().reshape(-1,1))
X_mod = clusterizer_mod.fit_predict(grouped_df_mod.to_numpy().reshape(-1,1))

# Generamos los labels
labels = pd.Series(X, index=grouped_df.index, name=('categorical', 'upz_cluster'))
labels_mod = pd.Series(X_mod, index=grouped_df_mod.index, name=('categorical', 'upz_cluster'))

# Se hace merge de los labels en las datas respectivas
df_clust = pd.merge(df_copy, labels, left_on = [('categorical', 'upz')], 
                    right_on=labels.index ,left_index=True,how='left')
df_clust.loc[:,('categorical', 'upz_cluster')] = df_clust[('categorical', 'upz_cluster')].map(lambda x: str(int(x)))
df_clust.reset_index(inplace = True,drop = True )

df_clust_mod = pd.merge(df_mod, labels_mod, left_on = [('categorical', 'upz')], 
                    right_on=labels_mod.index ,left_index=True,how='left')
df_clust_mod.loc[:,('categorical', 'upz_cluster')] = df_clust_mod[('categorical', 'upz_cluster')].map(lambda x: str(int(x)))
df_clust_mod.reset_index(inplace = True,drop = True )

In [ ]:
df_raw = pd.merge(df_raw, labels_mod, left_on = [('categorical', 'upz')], 
                    right_on=labels_mod.index ,left_index=True,how='left',validate='one_to_one')
df_raw.reset_index(inplace=True,drop=True)

In [ ]:
# Vemos los labels de cada data 
print('Sin filtrar: ',df_clust[('categorical','upz_cluster')].unique())
print('Filtrada: ',df_clust_mod[('categorical','upz_cluster')].unique())

In [ ]:
'''Aquí notamos un comportamiento extraño en la distribución de los valores
por categoría, nuestra hipótesis es que se debe a la potencial presencia de
outliers asociados a la variable 'price_per_m2'. Sin embargo al estar en escala 
logarítmica podemos observar una potencial diferencia en las medias por grupo,
para confirmar ésto, haremos un test oneway ANOVA sobre 'upz_cluster' vs 
'price_per_m2'.
'''
categoricalplot(df_clust,'upz_cluster',log=True)

In [ ]:
'''Se realiza el test oneway ANOVA para comprobar la diferencia de medias en la data sin filtrar
'''
idx = indexer(['price_per_m2','upz_cluster'], t_c=df_clust.columns)
grouped = df_clust[idx].groupby(idx[1])
total_groups = grouped.groups.keys()
groups = [group_cleaner(grouped.get_group(str(i)), 'upz_cluster', df_clust) for i in range(n_clust)]

F, p = f_oneway(*groups)
print('Estadistico F:',F)
print('p valor :', p)
alfa = 0.05
print('p<=alfa: ',p <= alfa)

In [ ]:
'''Veamos qué ocurre si consideramos las modificaciones realizadas en P2.2 a éste dataframe
'''
categoricalplot(df_clust_mod,'upz_cluster',log=False)

In [ ]:
''' Notamos que la mejora es considerable en cuanto a la identificación de price_per_m2 mediante
esta clusterización, además podemos observar que la cantidad de datos que se dropean con tal de
mejorar a ésta magnitud las representaciones e identificaciones solo corresponde a un 0.4% del total.
'''
diff = df_clust.shape[0]-df_clust_mod.shape[0]
perc_diff = 100 *(df_clust.shape[0]-df_clust_mod.shape[0])/df_clust.shape[0]
print('Luego del filtro como en P2.2, solo {} datos son dropeados, lo cual corresponde a un {}% de la cantidad total de datos'.format(diff,perc_diff))

In [ ]:
'''Se realiza el test oneway ANOVA con la data filtrada como en P2.2 
para comprobar la diferencia de medias. 
Ésto junto con lo expresado en las idenrificaciónes P2.2 creemos que es evidencia suficiente para seguir
trabajando con la data filtrada como en P2.2.
'''
idx = indexer(['price_per_m2','upz_cluster'], t_c=df_clust.columns)
grouped_mod = df_clust_mod[idx].groupby(idx[1])
total_groups_mod = grouped_mod.groups.keys()
groups_mod = [group_cleaner(grouped_mod.get_group(str(i)), 'upz_cluster', df_clust_mod) for i in range(n_clust)]

F, p = f_oneway(*groups_mod)
print('Estadistico F:',F)
print('p valor :', p)
alfa = 0.05
print('p<=alfa: ',p <= alfa)

## Parte 5

In [ ]:
# Actualizamos df con el clustering realizado en la parte anterior para la data filtrada
df = df_clust_mod.copy()

In [ ]:
'''Veamos ahora si existe dependencia entre variables de interés
'''
interest = ['price','surface','metrocuadrado_index','personas',
            'indice_inseguridad','price_per_m2',
            'n_rooms','n_garajes','n_bath']
idxs = indexer(interest)
idxs.sort()
idxs.remove(('numeric', 'price_per_m2'))
idxs.append(('numeric', 'price_per_m2'))
idxs

In [ ]:
data = df.reindex(idxs, axis=1).droplevel(0,axis=1).dropna()
sns.pairplot(data=data, diag_kind='kde')

In [ ]:
'''Veamos correlacion entre variables
'''
corrmatrix = df.corr()
corrmatrix

In [ ]:
col = indexer(['price_per_m2'])
corrmatrix[col].nlargest(20,col)

In [ ]:
'''
En primera instancia analizamos la correlación entre todas
las variables numéricas mediante un mapa de calor
(la varible objetivo están en la última fila).
'''
corrmat = df['numeric'].corr()
columnas = list(corrmat.columns)

corrmat = corrmat.reindex(index = columnas, columns = columnas)

fig, ax = plt.subplots(figsize=[14, 12])

sns.heatmap(corrmat, vmin=-.5, vmax=.9, linewidths=.01)

In [ ]:
'''
Se puede apreciar alta correlación (positiva y negativa)
entre algunos pares de variables. Para ubicarlos hacemos un
rearreglo 1D multi-índice y buscamos los que tengan
módulo más alto (distinto de 1).
'''
unoD=corrmat.stack()
unoD[unoD[unoD<1].abs().nlargest(20).index][::2]

In [ ]:
'''Se acordó un umbral de 0.7 para definir una correlación
potencialmente problemática en cuanto a la colinearidad.

En vista de lo anterior notamos una estrecha relación entre

- 'jovenes_14_24_anos_nini_perc'
- 'adultos_mayores_pobres_perc'
- 'indice_envegecimiento'

- 'personas'
- 'densidad_poblacion'

de las cuales elegimos una de cada grupo (la última) para
evitar colinearidad.

Cabe destacar que el par ('price','surface')
también tiene un valor muy alto, pero como estas variables
no se considerarán al momento de estimar 'price_per_m2' al
ser las generadoras de esta variable, no se estudia con
mayor detalle.

Por otro lado vemos que 'metrocuadrado_index'
tiene una alta correlación con la variable a explicar, por lo
que es un buen candidato para la selección final. Para ver cómo
se relacionan las otras variables con 'price_per_m2' analizamos
su columna.

Incluyendo ahora las variables que no presentan una correlación 
superior al umbral, tenemos a priori los siguientes candidatos
a ser incluidos en el modelo final:

- 'densidad_poblacion'
- 'indice_envegecimiento'
- 'jefe_mujer_perc' 
- 'areas_verdes_perc' 
- 'trabajoinf_ninos_5_17_anos_perc'
- 'trabajoinfampliado_ninos_5_17_anos_perc'
- 'indice_inseguridad'

'''
corrmat['price_per_m2'].sort_values(ascending = False)

In [ ]:
'''Tenemos las siguientes variables categóricas de interés

- 'upz_cluster'
- 'product_type'
- 'n_rooms'
- 'n_bath'
- 'n_garajes'
- 'furnished'

Buscamos ahora verificar estadísticamente si las variables generan
una diferencia significativa entre los grupos 'price_per_m2'.
Para ello haremos test oneway ANOVA.
'''

'''
Notamos que para cada variable considera se rechaza la nula,
por lo tanto cada variable presenta una separación estadística
considerable para 'price_per_m2'.
'''

from scipy.stats import f_oneway

# Recordemos que 'upz_cluster' y 'n_garajes' ya fueron validadas en P2.2 y P2.4 respectivamente
categoric_vars = ['furnished','product_type', 'n_rooms', 'n_bath']

for col in categoric_vars:
    print('{}:'.format(col))
    idx = indexer(['price_per_m2',col])
    grouped = df[idx].groupby(idx[1])
    total_groups = grouped.groups.keys()
    groups = [group_cleaner(grouped.get_group(i), col) for i in total_groups]
    
    F,p = f_oneway(*groups)
    print('Estadistico F:',F)
    print('p valor :', p)
    alpha = 0.05
    reject= p <= alpha
    if reject:
        print('Se rechaza la nula para {}'.format(col))
    else:
        print('NO se rechaza la nula para {}'.format(col))

In [ ]:
'''Nos interesa ahora ver si existe algun tipo de relación entre las variables
numéricas y categóricas que estamos considerando.
'''

In [ ]:
'''Procedemos a testear la significancia de cada variable en la
descripción de 'price_per_m2' utilizando el test t.
'''
from scipy.stats import ttest_ind
from scipy.stats import ks_2samp

numeric_vars = ['densidad_poblacion', 'indice_envegecimiento', 'jefe_mujer_perc', 
               'areas_verdes_perc', 'trabajoinf_ninos_5_17_anos_perc', 
               'trabajoinfampliado_ninos_5_17_anos_perc', 'indice_inseguridad']

ppm2 = np.array(df[('numeric','price_per_m2')])

alfa = 0.05
n = len(numeric_vars)
reject_matrix = np.zeros((n,n))
for i in range(n):
    var1 = numeric_vars[i]
    arr1 = np.array(df[('numeric',var1)].dropna())
    print(var1)
    for j in range(i+1,n):
        var2 = numeric_vars[j]
        arr2 = np.array(df[('numeric',var2)].dropna())
        #t,p = testt_ind(arr1,arr2,equal_var=False)
        t,p = ks_2samp(arr1,arr2)
        print('{}:\n Valor del estadístico: {}\n Valor de p: {}'.format(var2,t,p))
        reject = p < alfa
        if reject:
            reject_matrix[i][j] = 1
            #print('Se rechaza la nula, {} tiene significancia'.format(var))
            
reject_matrix

In [ ]:
from scipy.stats import ks_2samp
arr1=np.array(df[('numeric','jovenes_14_24_anos_nini_perc')].dropna())
arr2=np.array(df[('numeric','adultos_mayores_pobres_perc')].dropna())
t,p = ks_2samp(arr1,arr2)
print('Valor del estadístico: {}\n Valor de p: {}'.format(t,p))

## Parte 6

In [ ]:
'''Generamos un duplicado del df obtenido en la parte anterior
para generar ciertas transformaciones.
'''
#df_transformed = df_clust.copy()
df_transformed= df_clust_mod.copy()
df_transformed.isnull().sum()

In [ ]:


columnas = ['UPlArea','price','surface','metrocuadrado_index','personas',
           'trabajoinf_ninos_5_17_anos_perc','trabajoinfampliado_ninos_5_17_anos_perc',
           'jovenes_14_24_anos_nini_perc','indice_envegecimiento','jefe_mujer_perc',
            'adultos_mayores_pobres_perc','indice_inseguridad','areas_verdes_perc',
            'areas_verdes_perc','densidad_poblacion','price_per_m2',
            
            'n_garajes','upz_cluster','furnished','rent_type','n_rooms','product_type',
            'property_type','n_bath']

cols = indexer(columnas, t_c = df_transformed.columns)
df_transformed = df_transformed[cols]
df_transformed.isnull().sum()

In [ ]:
'''Debemos transformar algunas variables categoricas a labels según corresponda
'''
numerics= indexer(['UPlArea','price','surface','metrocuadrado_index','personas',
           'trabajoinf_ninos_5_17_anos_perc','trabajoinfampliado_ninos_5_17_anos_perc',
           'jovenes_14_24_anos_nini_perc','indice_envegecimiento','jefe_mujer_perc',
           'adultos_mayores_pobres_perc','indice_inseguridad','areas_verdes_perc',
           'areas_verdes_perc','densidad_poblacion','price_per_m2'], t_c = df_transformed.columns)
ordinales = indexer(['n_rooms','n_bath','n_garajes'], t_c = df_transformed.columns)
no_ordinales = indexer(['furnished','rent_type','property_type','product_type','upz_cluster'], t_c = df_transformed.columns)
#from sklearn.preprocessing import StandardScaler
#from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder

In [ ]:
data = df_transformed[numerics].dropna().copy()
idx = data.index
#scaler=StandardScaler()
#scaler=RobustScaler()
scaler=MinMaxScaler()
newdata = scaler.fit_transform(data)
newdata = pd.DataFrame(newdata, columns=pd.MultiIndex.from_tuples(numerics))
df_transformed.loc[:,numerics] = newdata

for col in ordinales:
    data = df_transformed[col].dropna().copy()
    idx = data.index
    enc = OrdinalEncoder()
    X = data.values.reshape([-1,1])
    transformed = enc.fit_transform(X)
    newcol = pd.Series(data = transformed.flatten(), index= idx)
    df_transformed.loc[idx,col] = newcol
    
for col in no_ordinales:
    data = df_transformed[col].dropna().copy()
    idx = data.index
    ohenc = OneHotEncoder(categories='auto',sparse=False)
    X = data.values.reshape([-1,1])
    transformed = ohenc.fit_transform(X)
    feature_names = ohenc.get_feature_names()
    feature_names = [name.replace('x0_','{}_'.format(col[1])) for name in feature_names]
    feature_names = [('categorical',name) for name in feature_names]
    newcols = pd.DataFrame(data = transformed, columns=pd.MultiIndex.from_tuples(feature_names), index= idx)
    df_transformed = df_transformed.drop(columns=col).join(newcols)

df_transformed.head(10)

In [ ]:
df_transformed.columns

In [ ]:
from sklearn.cluster import DBSCAN

outlier_detection = DBSCAN(min_samples = 2, eps = 0.8)

In [ ]:
# se clusteriza y se obtiene la proporción de outliers
X = df_transformed.to_numpy()
ol = outlier_detection.fit_predict(X)
(ol == -1).sum()/X.shape[0]

In [ ]:
# asignamos los clusters a una nueva columna y se mapea de forma que outlier=-1, inlier=1
df_clust_mod[('categorical', 'outlier')] = ol
data = df_clust_mod.copy()
data.columns = data.columns.droplevel()
data['outlier'] = data['outlier'].map(lambda x: 1 if x >=0 else -1)

# se crea una tabla de doble entrada para visualizar las distribuciones
kwargs = {'index': data['upz_cluster'], 'columns': data['outlier']}
table = pd.crosstab(**kwargs, margins=True)
table

In [ ]:
'''A primera vista es posible decir que en proporción, los outliers están distribuidos uniformemente 
a lo largo de los upz_clusters, es decir, no hay alguno de los clusters en particular que tienda a contener 
más outliers que los demás, sin embargo, aprovechando que las frecuencias observadas son mayores que 5
haremos un test chi 2 para asegurar esta inedependencia, si las frecuencias esperadas son también mayores que 5,
entonces se puede asegurar la confiabilidad del test
'''
from scipy.stats import chi2_contingency
chi2, p, dof, expected = chi2_contingency(table)
print(p < 0.01)
expected

In [ ]:
# repetimos el proceso esta vez viendo la distribución de outliers con respecto a product_type
kwargs['index'] = data['product_type']
table = pd.crosstab(**kwargs, margins=True)
table

In [ ]:
chi2, p, dof, expected = chi2_contingency(table)
print(p < 0.01)
expected

# P3

## Parte 1

In [ ]:
from sklearn.base import BaseEstimator,RegressorMixin

class RegresionBayesianaEmpirica(BaseEstimator,RegressorMixin):
    
    def __init__(self, alpha=0.01, beta=0.01, tol=1e-5, maxiter=200):
        self.alpha = alpha
        self.beta = beta
        #self.set_params(alpha=alpha_0,beta=beta_0)
        self.tol=tol
        self.maxiter = maxiter
        self.X = None
        self.y = None
    
    @property
    def X(self):
        return self.__X
    
    @property
    def y(self):
        return self.__y
    
    @X.setter
    def X(self, X):
        self.__X = X
#         if X.shape[0] == len(y):
#             self.__X = X
#         else:
#             raise ValueError('X debe tener la misma cantidad de filas que y')
        
    @y.setter
    def y(self,y):
        self.__y = y
#         if X.shape[0] == len(y):
#             self.__y = y
#         else:
#             raise ValueError('y debe tener la misma cantidad de filas que X')

    
    def get_posteriori(self, X, y, alpha, beta):
        S_n_inv = alpha * np.eye(X.shape[1]) + beta * X.T.dot(X)
        S_n = np.linalg.inv(S_n_inv)
        m_n = beta * S_n.dot(X.T).dot(y)
        return m_n, S_n
    
    def fit(self, X, y):
        # Se verifican las dimensiones
        if X.shape[0] == len(y):
            # Se asocia al objeto la data con la que se realizó el fit
            self.X = X
            self.y = y
        else:
            raise ValueError('la dimensión de y debe coincidir con la cantidad de filas que X')

        #Se inicializan las iteraciones y el diff 
        iterations=0
        diff = 2*self.tol
        # Se obtienen los parámetros alfa,beta y se calcular m_n, S_n en funcion de éstos
#         alfa, beta = self.get_params()['alpha'], self.get_params()['beta']
        alfa = self.alpha
        beta = self.beta
        propios=[np.real_if_close(val) for val in np.linalg.eig(beta*X.T.dot(X))[0]]
        
        while (diff >= self.tol) and (iterations <= self.maxiter):
        
            m_n, S_n = self.get_posteriori(X, y, alfa, beta)
        
            # Se calcula gamma en función de los valores anteriores
#             matrix = beta*X.T.dot(X)
#             [val/(alpha+val) for val in np.linalg.eig(matrix)][0]
            gamma = np.sum([val/(alpha+val) for val in propios])
            
            # Se calculan los nuevos alfa y beta
            new_alfa = gamma / (m_n.T.dot(m_n))
            
            new_beta = 1/(1/(X.shape[0]-gamma) * np.sum([(y[i]-m_n.T.dot(X[i,:]))**2 for i in range(X.shape[0])]))
            
            # Se guarda al cambio de los parámetros en norma l2
            diff = np.max([abs(alfa-new_alfa), abs(beta-new_beta)])
            
            # Se muestra en pantalla el estado actual
            if iterations%50==0:
                print('Iteración {}:\nalpha = {}\nbeta = {}\n'.format(iterations,new_alfa,new_beta))
            
            # Se fijan los nuevos parámetros
            #self.set_params(alpha=new_alfa, beta=new_beta)
            alfa = new_alfa
            beta = new_beta
                
            # Se sigue iterando
            iterations+=1
        
        self.alpha = alfa
        self.beta = beta
        print('Fit Terminado en la iteración {}, con diferencias entre actualizaciones (en norma 2) {}'.format(iterations,diff))
        return self
    
    def predict(self,X_,return_std=False):
        m_n, S_n = self.get_posteriori(self.X,self.y,self.alpha,self.beta)
        
        y_ = X_.dot(m_n)
        #y_std = 1/beta + x.T.dot(S_n).dot(x)
        y_std = []
        for i in range(len(X_)):
            x = X_[i]
            sigma2_n = 1/self.beta + x.T.dot(S_n).dot(x)
            y_std.append(np.sqrt(sigma2_n))
        
        if return_std:
            return y_, y_std
        else:
            return y_


## Parte 2

In [ ]:
'''Comenzamos realizando un test de normalidad sobre las variables numéricas dado que los valóres más extremos fueron
removidos por los argumentos presentados en anteriores preguntas. De esta forma se decidirá cuales de las variables numéricas
con aptas para usar StandardScaler.'''

from scipy.stats import normaltest as nt
print(df_raw['categorical'].columns)
s, p = nt(df_raw['numeric'], axis=0)
p<0.05

In [ ]:
'''Creamos una copia de la data a trabajar, dropeando las variables miscelaneas 
junto con `price`, `surface`, `property_type` y `upz`.
Además dropeamos el nivel más externo del multiíndice por comodidad.'''

d = df_raw.drop(columns='miscelaneous', level=0).drop(columns=['price', 'surface', 'property_type', 'upz'],level=1).copy()
d.columns = d.columns.droplevel()
d.n_garajes.unique()



In [ ]:
"""Definimos ahora listas de variables acorde al tratamiento a darles. Dados los resultados del test de normalidad,
todas las variables serán consideradas NO provenientes de una normal. Es por esta razón que no se hará uso del objeto 
StandardScaler."""
categorical_vars = ['product_type','rent_type', 'upz_cluster', 'furnished']

ordinal_vars = ['n_rooms', 'n_bath', 'n_garajes']
ordinal_categories = [['1', '2', '3', '4', '5', '5+'],
                      ['1', '2', '3', '4', '5', '5+'],
                      ['1', '2', '3', '4', '4+']
                     ]
numeric_vars = ['UPlArea', 'metrocuadrado_index', 'personas',
       'trabajoinf_ninos_5_17_anos_perc',
       'trabajoinfampliado_ninos_5_17_anos_perc',
       'jovenes_14_24_anos_nini_perc', 'indice_envegecimiento',
       'jefe_mujer_perc', 'adultos_mayores_pobres_perc', 'indice_inseguridad',
       'areas_verdes_perc', 'densidad_poblacion']

In [ ]:
# Se hacen los imports necesarios para las transformaciones
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder, OrdinalEncoder, PolynomialFeatures

In [ ]:
# Se generan los pipelines para los distintos tipos de variables

# pipeline variables categóricas
pipe_categorico = Pipeline(steps=[('uno_caliente_codificador', OneHotEncoder(sparse=False))])

# pipeline variables ordinales
pipe_ordinal = Pipeline(steps=[('imputador_ordinal', SimpleImputer(strategy='most_frequent')),
                              ('ordinal_codificador', OrdinalEncoder(categories=ordinal_categories))])

# pipeline numéricas (todas NO provenientes de una normal)
pipe_numerico = Pipeline(steps=[('min_max_escalador', MinMaxScaler()),
                               ('poly_features', PolynomialFeatures(degree=3))])

# pipeline final
pipe_shishigang = ColumnTransformer(transformers=[
    ('pipe_categotico', pipe_categorico, categorical_vars),
    ('pipe_ordinal', pipe_ordinal, ordinal_vars),
    ('pipe_numerico', pipe_numerico, numeric_vars)
])

In [ ]:
X = d.drop(columns='price_per_m2').copy()
X_t = pipe_shishigang.fit_transform(X)

In [ ]:
X_t.shape

In [ ]:
'''Probemos la P1 con lo anterior'''
y_t=d['price_per_m2'].values
estimador=RegresionBayesianaEmpirica(maxiter=50)
estimador.fit(X_t[:100],y_t[:100])

In [ ]:
y_pred,y_std=estimador.predict(X_t[100:110],return_std=True)

## Parte 3

In [ ]:
ColumnTransformer(transformers=[
    ('pipe_categotico', pipe_categorico, categorical_vars),
    ('pipe_ordinal', pipe_ordinal, ordinal_vars),
    ('pipe_numerico', pipe_numerico, numeric_vars)
])

In [ ]:
X = d[(d['price_per_m2']>1)&(d['price_per_m2']<10**5)].drop(columns='price_per_m2').copy()
y = d[(d['price_per_m2']>1)&(d['price_per_m2']<10**5)].price_per_m2.values.copy()

In [ ]:
#from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [ ]:
pipe = Pipeline([('transformador', pipe_shishigang), ('regresor', RegresionBayesianaEmpirica(alpha=10**(-7),beta=10**(-5)))])

In [ ]:
#pipe['regresor'].predict(X_t[-645:-644])
# x=X_t[-645:-644][0]
# mean = m_n.T.dot(x)
# sigma2_n = 1/beta + x.T.dot(S_n).dot(x)
# print(sigma2_n)

In [ ]:
pipe.fit(X_train,y_train)

In [ ]:
pipe['regresor'].get_params()

In [ ]:
y_pred, y_std = pipe.predict(X_test,return_std=True)
from sklearn.metrics import mean_squared_error
rms = np.sqrt(mean_squared_error(y_test, y_pred))
rms

In [ ]:
np.std(y_test)

In [ ]:
pipe.score(X_test,y_test)

In [ ]:
np.sum([x<0 for x in y_std])